<a href="https://colab.research.google.com/github/pasumarthi/EVA/blob/master/Session25/Assignment_cars_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving resized_96.zip to resized_96.zip


In [3]:
!ls

resized_96.zip	sample_data


In [4]:
!unzip resized_96.zip

Archive:  resized_96.zip
   creating: resized_96/
  inflating: resized_96/0001_resized.jpg  
  inflating: resized_96/0002_resized.jpg  
  inflating: resized_96/0004_resized.jpg  
  inflating: resized_96/0005_resized.jpg  
  inflating: resized_96/0007_resized.jpg  
  inflating: resized_96/0008_resized.jpg  
  inflating: resized_96/0011_resized.jpg  
  inflating: resized_96/0013_resized.jpg  
  inflating: resized_96/0014_resized.jpg  
  inflating: resized_96/0015_resized.jpg  
  inflating: resized_96/0016_resized.jpg  
  inflating: resized_96/0017_resized.jpg  
  inflating: resized_96/0019_resized.jpg  
  inflating: resized_96/0020_resized.jpg  
  inflating: resized_96/0021_resized.jpg  
  inflating: resized_96/0022_resized.jpg  
  inflating: resized_96/0023_resized.jpg  
  inflating: resized_96/0024_resized.jpg  
  inflating: resized_96/0025_resized.jpg  
  inflating: resized_96/0028_resized.jpg  
  inflating: resized_96/0029_resized.jpg  
  inflating: resized_96/0030_resized.jpg  
  in

In [0]:
!mkdir images

In [0]:
# !rm -r images

In [7]:
from __future__ import print_function, division

#from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import glob
from PIL import *

import numpy as np


Using TensorFlow backend.


In [0]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 96
        self.img_cols = 96
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 3 * 3, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((3, 3, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        #(X_train, _), (_, _) = mnist.load_data()

        filelist = glob.glob("resized_96/*")
        #print("filelist",list(filelist))
        cars_tensor = np.array([np.array(Image.open(fname)) for fname in filelist])

        print("------------>",cars_tensor.shape)

        '''# Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)'''

        cars_tensor = cars_tensor / 127.5 - 1.

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, cars_tensor.shape[0], batch_size)
            imgs = cars_tensor[idx];print("-------->",imgs.shape)

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/car%d.png" % epoch)
        plt.close()

In [9]:
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=10000, batch_size=32, save_interval=50)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 25, 25, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 25, 25, 64)        256  

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.796354, acc.: 20.31%] [G loss: 0.376523]
--------> (32, 96, 96, 3)
1 [D loss: 0.791013, acc.: 57.81%] [G loss: 1.836118]
--------> (32, 96, 96, 3)
2 [D loss: 0.666096, acc.: 64.06%] [G loss: 2.581069]
--------> (32, 96, 96, 3)


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


3 [D loss: 0.989474, acc.: 40.62%] [G loss: 2.246336]
--------> (32, 96, 96, 3)
4 [D loss: 0.948735, acc.: 37.50%] [G loss: 1.767388]
--------> (32, 96, 96, 3)
5 [D loss: 0.787621, acc.: 57.81%] [G loss: 1.708993]
--------> (32, 96, 96, 3)
6 [D loss: 0.932374, acc.: 43.75%] [G loss: 1.654977]
--------> (32, 96, 96, 3)
7 [D loss: 0.515799, acc.: 79.69%] [G loss: 1.330406]
--------> (32, 96, 96, 3)
8 [D loss: 0.484099, acc.: 73.44%] [G loss: 1.745954]
--------> (32, 96, 96, 3)
9 [D loss: 0.405891, acc.: 82.81%] [G loss: 1.758890]
--------> (32, 96, 96, 3)
10 [D loss: 0.441440, acc.: 76.56%] [G loss: 2.107323]
--------> (32, 96, 96, 3)
11 [D loss: 0.436805, acc.: 79.69%] [G loss: 2.319931]
--------> (32, 96, 96, 3)
12 [D loss: 0.844405, acc.: 54.69%] [G loss: 2.056671]
--------> (32, 96, 96, 3)
13 [D loss: 0.972743, acc.: 56.25%] [G loss: 2.352290]
--------> (32, 96, 96, 3)
14 [D loss: 0.751001, acc.: 56.25%] [G loss: 1.401151]
--------> (32, 96, 96, 3)
15 [D loss: 0.509613, acc.: 79.69%]